In [51]:
pip install pecanpy.graph

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pecanpy.graph (from versions: none)
ERROR: No matching distribution found for pecanpy.graph


In [50]:
from pecanpy.graph import SparseOTF

ImportError: cannot import name 'SparseOTF' from 'pecanpy.graph' (C:\Users\Vaibhav\miniconda3\Lib\site-packages\pecanpy\graph.py)

In [57]:
import pandas as pd
import os
import gc

from pecanpy import pecanpy as node2vec

def Runnode2vec(filepath):
    n2v = node2vec.SparseOTF(p=1, q=1, workers=2, verbose=True)

    if not os.path.isfile(filepath):
        print(f"Error: File '{filepath}' not found or is not accessible.")
        return None

    try:
        edge_list = n2v.read_edg(filepath, weighted=False, directed=False)
    except FileNotFoundError:
        print(f"Error: Edge list file '{filepath}' could not be read.")
        del n2v
        gc.collect()
        return None
    except Exception as e:
        print(f"Error reading edge list from '{filepath}': {e}")
        del n2v
        gc.collect()
        return None

    try:
        emd = n2v.embed(dim=128, num_walks=10, walk_length=80, window_size=10, epochs=1)
    except IndexError:
        print("Error: Index out of range while embedding.")
        del n2v
        gc.collect()
        return None
    except Exception as e:
        print(f"Error embedding the edge list from '{filepath}': {e}")
        del n2v
        gc.collect()
        return None

    try:
        n2v_emd = pd.DataFrame(emd, n2v.IDlst)
        n2v_emd.columns = ['network_' + str(col) for col in n2v_emd.columns]
        print(n2v.nodes)
    except Exception as e:
        print(f"Error while creating the pandas DataFrame: {e}")
        return None

    del n2v
    gc.collect()
    return n2v_emd.reset_index().rename(columns={"index": "ensembl"})

network = Runnode2vec(r"ESCA\eso_fin_-fgin_edge_list.edg")

if network is not None:
    master = master.merge(network, on='ensembl')
    master.to_csv(r"ESCA\node_node2vec_data.csv", index=None)

  0%|                                                                                         | 0/6440 [00:00<…

Error while creating the pandas DataFrame: 'SparseOTF' object has no attribute 'IDlst'


In [58]:
from pecanpy import pecanpy as node2vec
def Runnode2vec(filepath):
    n2v = node2vec.SparseOTF(p=1, q=1, workers=4, verbose=True)

    edge_list = n2v.read_edg(filepath, weighted=False, directed=False)
    emd = n2v.embed(dim=128, num_walks=10, walk_length=80, window_size=10, epochs=10)

    n2v_emd = pd.DataFrame(emd, n2v.IDlst)

    n2v_emd.columns = ['network_' + str(col) for col in n2v_emd.columns]

    return n2v_emd.reset_index().rename(columns={"index":"ensembl"})

In [1]:
import os
os.chdir('D:/gene prioritization/genedragnn/')
cwd = os.getcwd()
cwd

'D:\\gene prioritization\\genedragnn'

In [2]:
import pandas as pd

def ImportDGN():
    dgn = pd.read_csv("ESCA/C0279626_disease_gda_summary.tsv",sep='\t')
    dgn_dict = pd.read_csv("ESCA/gda_dictionary.csv", index_col=None)

    score_threshold = 0.02
    ei_threshold = 0.7

    dgn = dgn[['Gene', 'EI_gda', 'Score_gda']]
    dgn = dgn.loc[dgn['Score_gda'] >= score_threshold]
    dgn = dgn.loc[dgn['EI_gda'] > ei_threshold]
    dgn.rename({'Score_gda':'gda_score'}, axis=1, inplace=True)
    dgn = dgn.merge(dgn_dict, on="Gene").drop(['Gene'], axis=1)
    dgn['gda_score'] = 1

    return dgn[['ensembl', 'gda_score']]

In [2]:
import pandas as pd
import re

def ImportGDC(file_path):
    gdc = pd.read_csv(file_path)

    feature_name_columns = ['# SSM Affected Cases in Cohort', '# CNV Gain', '# CNV Loss']
    
    for feature_column in feature_name_columns:
        split_columns = gdc[feature_column].replace({',':''}, regex=True).str.split(' ', n=2, expand=True)
        gdc[f'{feature_column}_1'] = split_columns[0].astype(float)
        gdc[f'{feature_column}_2'] = split_columns[1]
        
        # Extract numerical part and convert to float
        gdc[f'{feature_column}_3'] = split_columns[2].replace('[^0-9.]', '', regex=True).astype(float)
        
        gdc[feature_column] = gdc[f'{feature_column}_1'] / gdc[f'{feature_column}_3']
        gdc = gdc.drop([f'{feature_column}_1', f'{feature_column}_2', f'{feature_column}_3'], axis=1)

    gdc.drop(['Symbol', 'Name', 'Cytoband', 'Type', 'Annotations', 'Survival'], axis=1, inplace=True)

    split_columns = gdc['# SSM Affected Cases Across the GDC'].replace({',':''}, regex=True).str.split(' ', n=2, expand=True)
    gdc['1'] = split_columns[0].astype(float)
    gdc['2'] = split_columns[1]
    
    # Extract numerical part and convert to float
    gdc['3'] = split_columns[2].replace('[^0-9.]', '', regex=True).astype(float)
    
    gdc['# SSM Affected Cases Across the GDC'] = gdc['1'] / gdc['3']
    gdc = gdc.drop(['1', '2', '3'], axis=1)

    gdc = gdc.rename({'# SSM Affected Cases in Cohort': 'nih_ssm_in_cohort',
                      '# SSM Affected Cases Across the GDC': 'nih_ssm_across_gdc',
                      '# CNV Gain': 'nih_cnv_gain',
                      '# CNV Loss': 'nih_cnv_loss',
                      'Gene ID': 'ensembl',
                      '#Mutations': 'nih_tot_mutations'}, axis=1)

    return gdc


In [20]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

def ImportHPA():
    hpa = pd.read_csv(r"data/raw_data/hpa_gene_features.tsv",sep='\t').drop_duplicates(subset='Gene')

    identifiers = [
        "Gene",
        "Ensembl",
        "Uniprot"
    ]
    discrete_features = [
        "Protein class",
        "Biological process",
        "Molecular function",
        "Disease involvement",
        "Subcellular location",
    ]
    continuous_features = [
        "Tissue RNA - esophagus [NX]",
        "Single Cell Type RNA - Mucus-secreting cells [NX]"
        
    ]

    hpa_features = hpa.iloc[:, hpa.columns.isin(identifiers+discrete_features+continuous_features)]

    hpa_features["Tissue RNA - esophagus [NX]"] = (hpa_features["Tissue RNA - esophagus [NX]"] - hpa_features["Tissue RNA - esophagus [NX]"].mean()) / hpa_features["Tissue RNA - esophagus [NX]"].std()
    col = "Single Cell Type RNA - Mucus-secreting cells [NX]"
    hpa_features[col] = (hpa_features[col] - hpa_features[col].mean()) / hpa_features[col].std()

    def explode(feature) :
        return feature.apply(lambda x: x.replace(' ', '').split(','))

    hpa_clean = hpa.fillna('')
    for ft in discrete_features :
        hpa_clean[ft] = explode(hpa_clean[ft])

    protein_class = hpa_clean["Protein class"].explode().unique()
    biological_process = hpa_clean["Biological process"].explode().unique()
    molecular_function = hpa_clean["Molecular function"].explode().unique()
    disease_involvement = hpa_clean["Disease involvement"].explode().unique()
    subcellular_location = hpa_clean["Subcellular location"].explode().unique()
    GO_features = np.concatenate([protein_class, biological_process, molecular_function, disease_involvement, subcellular_location])

    RowFeatures = pd.DataFrame(data = 0,index = hpa_clean['Ensembl'],columns=GO_features)
    counter = 0

    for index, row in RowFeatures.iterrows() :
        features = hpa_clean.iloc[counter][['Protein class', 'Biological process', 'Molecular function', 'Disease involvement', 'Subcellular location']].to_list()
        flattened = [item for sublist in features for item in sublist if item]
        for t in flattened :
            row[t] = 1
        counter +=1 

    n_comp = 100
    svd = TruncatedSVD(n_components = n_comp)
    svdModel = svd.fit(RowFeatures)
    visits_emb = svdModel.transform(RowFeatures)
    hpa = pd.DataFrame(data=visits_emb, index=RowFeatures.index).reset_index()
    hpa.columns = ['hpa_' + str(col) for col in hpa.columns]
    hpa = hpa.rename({'Tissue RNA - esophagus [NX]':'nx_tissue_rna_esophagus ', 'hpa_Single Cell Type RNA - Mucus-secreting cells [NX]':'nx_sing_cell_type_mucus_secreting_cells', 
        'hpa_Ensembl':'ensembl','hpa_Uniprot':'uniprot'}, axis=1)

    return hpa




In [4]:
import pandas as pd

def remove_redun(el, verbose=False):
    if verbose:
        print("Original Size: ", len(el))

    el_new = el.iloc[:, 0:2].apply(sorted, axis=1)

    el_new = pd.DataFrame.from_dict(dict(zip(el_new.index, el_new.values))).T 

    el_new = el_new.drop_duplicates()
    if verbose:
        postDrop = len(el_new)
        print("After Dropping Duplicates: ", len(el_new), "(-", len(el)-postDrop, ")")

    el_new = el_new.merge(el, left_on=[el_new.columns[0],el_new.columns[1]],  right_on=[el.columns[0], el.columns[1]])
    if verbose:
        print("After Merging: ", len(el_new), "(-", postDrop-len(el_new), ")")
        print()

    return el_new.iloc[:, 2:]

def map_IDs(el, gmap, verbose = False, dropNaNvalues = True):
    gp_map_f = gmap.set_index('#string_protein_id')['alias']

    el_converted = el.reset_index(drop=True)

    el_converted[el_converted.columns[0]] = el_converted[el_converted.columns[0]].map(gp_map_f)
    el_converted[el_converted.columns[1]] = el_converted[el_converted.columns[1]].map(gp_map_f)

    if verbose:
        print("NaN values per Column:", el_converted[el_converted.columns[0]].isna().sum(), el_converted[el_converted.columns[1]].isna().sum())

    if dropNaNvalues:
        el_converted = el_converted.dropna()
        if verbose:
            print("New edge list size:", len(el_converted), "( -", len(el)-len(el_converted), ")")
    
    return el_converted

def ImportSTRING():
    el_map = pd.read_csv(r"data/raw_data/9606.protein.aliases.v11.5.txt", sep="\t")
    el = pd.read_csv(r"data/raw_data/9606.protein.links.detailed.v11.5.txt", sep=" ")
    el_map = el_map.loc[el_map.source == 'Ensembl_gene']

    el = remove_redun(el, True)
    el = map_IDs(el, el_map, verbose=True)

    return el



In [5]:
# Process the gdc dataset to include only desired columns and scores
gdc = ImportGDC('ESCA/gdc_esca_fin.csv')
gdc

,ensembl,nih_ssm_in_cohort,nih_ssm_across_gdc,nih_cnv_gain,nih_cnv_loss,# Mutations
0,ENSG00000141510,0.008414,0.300396,0.001396,0.003457,116
1,ENSG00000181143,0.002979,0.175983,0.008341,0.002900,73
2,ENSG00000168702,0.002207,0.108504,0.002399,0.001340,53
3,ENSG00000164796,0.001987,0.120939,0.006069,0.005014,43
4,ENSG00000167548,0.001325,0.081256,0.001953,0.001340,27
...,...,...,...,...,...,...
697,ENSG00000183765,0.000000,0.010058,0.001284,0.002900,0
698,ENSG00000164985,0.000000,0.009509,0.001340,0.004680,0
699,ENSG00000136504,0.000000,0.009083,0.003624,0.007233,0
700,ENSG00000132475,0.000000,0.004511,0.003346,0.001228,0


In [21]:
hpa = ImportHPA()
hpa
#hpa.to_csv('ESCA/hpa_features_v1.csv')

C:\Users\Vaibhav\AppData\Local\Temp\ipykernel_8968\455513766.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hpa_features["Tissue RNA - esophagus [NX]"] = (hpa_features["Tissue RNA - esophagus [NX]"] - hpa_features["Tissue RNA - esophagus [NX]"].mean()) / hpa_features["Tissue RNA - esophagus [NX]"].std()
C:\Users\Vaibhav\AppData\Local\Temp\ipykernel_8968\455513766.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hpa_features[col] = (hpa_features[col] - hpa_features[col].mean()) / hpa_features[col].

,ensembl,hpa_0,hpa_1,hpa_2,hpa_3,hpa_4,hpa_5,hpa_6,hpa_7,hpa_8,...,hpa_90,hpa_91,hpa_92,hpa_93,hpa_94,hpa_95,hpa_96,hpa_97,hpa_98,hpa_99
0,ENSG00000175899,0.443505,0.313574,0.430307,-0.285337,-0.351412,1.545059,-0.450936,-0.444200,-0.116774,...,0.088608,0.067989,-0.194712,0.120353,0.021725,0.294454,0.080320,0.650595,0.517039,0.260803
1,ENSG00000128274,0.532289,0.789331,-0.122159,0.019000,1.427162,-0.065659,0.001669,0.386983,-0.028900,...,-0.017782,-0.266771,-0.022111,0.124842,0.050235,0.100747,-0.029940,-0.063440,0.043904,0.044050
2,ENSG00000094914,1.605229,0.936681,0.345886,0.183699,-1.001764,-0.846346,0.550995,0.368205,0.202731,...,0.309754,-0.148516,0.013264,-0.331476,-0.280180,-0.156086,0.186635,-0.018856,0.114130,-0.319874
3,ENSG00000081760,0.467196,0.690415,-0.066723,0.335478,1.017124,0.029579,-0.131714,0.212387,0.075120,...,-0.065055,0.279989,0.060622,-0.084932,0.104864,-0.036440,-0.023233,0.131144,-0.159145,-0.206964
4,ENSG00000114771,0.956718,0.202132,-0.490047,-0.524460,0.467132,-0.207775,-0.494652,0.192615,0.077836,...,-0.004260,0.002651,0.000884,-0.004813,-0.000156,0.005321,-0.005672,-0.006520,0.004980,-0.022990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15008,ENSG00000198455,1.436813,-1.455513,0.510199,0.224208,0.323736,-0.171141,-0.060697,0.088626,0.103071,...,-0.043788,-0.042161,-0.025889,-0.011543,-0.013621,0.004348,0.031256,0.008747,-0.006570,-0.010959
15009,ENSG00000070476,1.140170,-1.162868,0.551409,0.127004,0.119115,-0.120434,-0.421260,0.798186,0.041917,...,-0.032481,-0.042760,-0.006132,-0.003529,-0.004581,0.006094,0.033696,0.021716,-0.018482,-0.020511
15010,ENSG00000162378,0.782655,-0.057857,-0.397211,0.017498,-0.135123,-0.248882,-0.438176,0.113363,-0.295955,...,0.013299,-0.021881,0.010280,0.021485,0.009559,-0.054986,0.029477,-0.007672,0.043851,0.023778
15011,ENSG00000159840,1.042306,0.183392,-0.327697,0.065425,-0.459893,0.507171,-0.610148,0.067066,0.368216,...,0.369714,-0.066808,-0.318255,-0.123101,0.432497,0.090390,0.165978,-0.099273,0.043760,0.103226


In [24]:
hpa_org = pd.read_csv(r"data/raw_data/hpa_gene_features.tsv",sep='\t').drop_duplicates(subset='Gene')
merged_df = pd.merge(hpa[['ensembl']], hpa_org[['Ensembl', 'Uniprot']], left_on='ensembl', right_on='Ensembl', how='left')
merged_df.rename(columns={'Uniprot': 'uniprot'}, inplace=True)
df1 = pd.concat([hpa, merged_df['uniprot']], axis=1)
df1.to_csv("ESCA/uniprot_hpa_v2.csv")

In [7]:
master = hpa.merge(gdc, on="ensembl")
master

,ensembl,hpa_0,hpa_1,hpa_2,hpa_3,hpa_4,hpa_5,hpa_6,hpa_7,hpa_8,...,hpa_95,hpa_96,hpa_97,hpa_98,hpa_99,nih_ssm_in_cohort,nih_ssm_across_gdc,nih_cnv_gain,nih_cnv_loss,# Mutations
0,ENSG00000136754,1.224354,0.375444,0.415797,-0.193960,-0.575138,0.628131,-0.201646,-0.070530,-1.111441,...,0.006600,-0.009743,-0.039397,0.016256,-0.023728,0.000552,0.005669,0.002789,0.001953,1
1,ENSG00000097007,2.301717,0.496194,0.934254,-1.010038,0.761272,0.521151,0.233309,-0.609991,-1.182277,...,0.226422,-0.526213,0.498607,-0.089380,-0.003785,0.001656,0.015849,0.002845,0.002343,3
2,ENSG00000143322,1.687926,0.165574,-0.283591,-0.589243,0.995969,0.729554,-0.216856,-0.573510,-1.171009,...,0.363771,-0.400097,-0.004601,0.093254,-0.092920,0.000000,0.018226,0.003846,0.007787,0
3,ENSG00000144476,0.672661,0.740794,0.482869,1.095246,0.375547,1.387898,-0.059448,-0.018398,-1.061311,...,-0.003404,-0.072961,-0.076042,0.027603,-0.027359,0.000000,0.009326,0.001730,0.002120,0
4,ENSG00000123983,1.325595,0.596590,-0.036726,0.099112,0.679536,0.678640,-0.449801,0.092542,-0.877696,...,-0.052686,0.042310,-0.187494,-0.170062,0.325041,0.001104,0.007498,0.001953,0.001953,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,ENSG00000130844,1.703319,-1.538011,0.976639,0.103362,0.270447,0.698892,-0.142706,-0.054580,-0.634387,...,0.007948,-0.020601,-0.000828,0.000044,-0.027276,0.000552,0.012618,0.002622,0.001451,1
640,ENSG00000126746,1.703319,-1.538011,0.976639,0.103362,0.270447,0.698892,-0.142706,-0.054580,-0.634387,...,0.007948,-0.020601,-0.000828,0.000044,-0.027276,0.001656,0.007985,0.003012,0.001507,3
641,ENSG00000197013,1.503480,-1.626249,0.635482,0.246415,0.371925,-0.183920,-0.085103,0.171952,0.154054,...,0.005915,-0.003141,0.015359,0.001862,0.001211,0.002758,0.012130,0.002399,0.001619,5
642,ENSG00000198795,2.082088,-1.458554,1.708238,-0.042659,0.100264,0.606444,0.057277,0.026743,-0.745850,...,-0.079954,0.064610,-0.047011,-0.021797,-0.002556,0.006057,0.032673,0.002288,0.003401,12


In [8]:
#Import STRING Dataset

el = ImportSTRING()
el

Original Size:  11938498
After Dropping Duplicates:  5969249 (- 5969249 )
After Merging:  5969249 (- 0 )

NaN values per Column: 0 0
New edge list size: 5969249 ( - 0 )


,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score
0,ENSG00000004059,ENSG00000154678,0,0,0,54,0,0,144,155
1,ENSG00000004059,ENSG00000180370,0,0,0,0,180,0,61,197
2,ENSG00000004059,ENSG00000100228,0,0,0,62,152,0,101,222
3,ENSG00000004059,ENSG00000107263,0,0,0,0,161,0,64,181
4,ENSG00000004059,ENSG00000184900,0,0,0,82,213,0,72,270
...,...,...,...,...,...,...,...,...,...,...
5969244,ENSG00000279723,ENSG00000172381,0,0,0,159,0,0,0,159
5969245,ENSG00000279723,ENSG00000177853,0,0,0,0,0,0,181,181
5969246,ENSG00000268606,ENSG00000269405,0,0,0,76,0,0,283,309
5969247,ENSG00000223802,ENSG00000261832,0,0,0,0,0,0,218,218


In [9]:
el_allgenes = pd.concat([el['protein1'], el['protein2']]).drop_duplicates()
master = master.loc[master['ensembl'].isin(el_allgenes)]
master

,ensembl,hpa_0,hpa_1,hpa_2,hpa_3,hpa_4,hpa_5,hpa_6,hpa_7,hpa_8,...,hpa_95,hpa_96,hpa_97,hpa_98,hpa_99,nih_ssm_in_cohort,nih_ssm_across_gdc,nih_cnv_gain,nih_cnv_loss,# Mutations
0,ENSG00000136754,1.224354,0.375444,0.415797,-0.193960,-0.575138,0.628131,-0.201646,-0.070530,-1.111441,...,0.006600,-0.009743,-0.039397,0.016256,-0.023728,0.000552,0.005669,0.002789,0.001953,1
1,ENSG00000097007,2.301717,0.496194,0.934254,-1.010038,0.761272,0.521151,0.233309,-0.609991,-1.182277,...,0.226422,-0.526213,0.498607,-0.089380,-0.003785,0.001656,0.015849,0.002845,0.002343,3
2,ENSG00000143322,1.687926,0.165574,-0.283591,-0.589243,0.995969,0.729554,-0.216856,-0.573510,-1.171009,...,0.363771,-0.400097,-0.004601,0.093254,-0.092920,0.000000,0.018226,0.003846,0.007787,0
3,ENSG00000144476,0.672661,0.740794,0.482869,1.095246,0.375547,1.387898,-0.059448,-0.018398,-1.061311,...,-0.003404,-0.072961,-0.076042,0.027603,-0.027359,0.000000,0.009326,0.001730,0.002120,0
4,ENSG00000123983,1.325595,0.596590,-0.036726,0.099112,0.679536,0.678640,-0.449801,0.092542,-0.877696,...,-0.052686,0.042310,-0.187494,-0.170062,0.325041,0.001104,0.007498,0.001953,0.001953,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,ENSG00000130844,1.703319,-1.538011,0.976639,0.103362,0.270447,0.698892,-0.142706,-0.054580,-0.634387,...,0.007948,-0.020601,-0.000828,0.000044,-0.027276,0.000552,0.012618,0.002622,0.001451,1
640,ENSG00000126746,1.703319,-1.538011,0.976639,0.103362,0.270447,0.698892,-0.142706,-0.054580,-0.634387,...,0.007948,-0.020601,-0.000828,0.000044,-0.027276,0.001656,0.007985,0.003012,0.001507,3
641,ENSG00000197013,1.503480,-1.626249,0.635482,0.246415,0.371925,-0.183920,-0.085103,0.171952,0.154054,...,0.005915,-0.003141,0.015359,0.001862,0.001211,0.002758,0.012130,0.002399,0.001619,5
642,ENSG00000198795,2.082088,-1.458554,1.708238,-0.042659,0.100264,0.606444,0.057277,0.026743,-0.745850,...,-0.079954,0.064610,-0.047011,-0.021797,-0.002556,0.006057,0.032673,0.002288,0.003401,12


In [10]:
el_intersect = el.iloc[:, :2].merge(master["ensembl"], right_on="ensembl", left_on='protein1').drop("ensembl", axis=1)
el_intersect = el_intersect.merge(master["ensembl"], right_on="ensembl", left_on='protein2').drop("ensembl", axis=1).rename(columns={'protein1':'gene1', 'protein2':'gene2'})
el = el_intersect.merge(el, right_on=['protein1', 'protein2'], left_on=['gene1', 'gene2']).drop(['protein1','protein2'], axis=1)
el

,gene1,gene2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score
0,ENSG00000019582,ENSG00000181690,0,0,0,0,56,0,210,222
1,ENSG00000100030,ENSG00000181690,0,0,0,55,133,0,59,161
2,ENSG00000110092,ENSG00000181690,0,0,0,0,56,0,183,195
3,ENSG00000111276,ENSG00000181690,0,0,0,0,58,0,158,173
4,ENSG00000114354,ENSG00000181690,0,0,0,0,0,0,170,169
...,...,...,...,...,...,...,...,...,...,...
51578,ENSG00000110777,ENSG00000168092,0,0,0,0,0,0,223,222
51579,ENSG00000067560,ENSG00000168092,0,0,0,82,611,0,0,627
51580,ENSG00000120457,ENSG00000168092,0,0,0,0,0,0,168,167
51581,ENSG00000175643,ENSG00000261652,0,0,0,0,0,0,166,165


In [16]:

#el[['gene1', 'gene2']].to_csv('ESCA/eso_fin_-fgin_edge_list.edg', sep="\t", index=False, header=False) 
#el.to_csv('ESCA/esca_fin-fgin_edge_list_features.edg', index=False) 
el1=el[['gene1', 'gene2']]
np.save('ESCA/esca_fin-fgin_edge_list_features.npy', el[['gene1', 'gene2']])

In [13]:
dgn = ImportDGN()
dgn = dgn.loc[dgn['ensembl'].isin(master['ensembl'])]
dgn.to_csv("ESCA\dgn_fin.csv")


In [14]:
network = pd.read_csv("ESCA/eso_node2vecdata.csv")
master = master.merge(network, on='ensembl')
master.to_csv("ESCA/node_node2vec_data.csv", index=None)
master

,ensembl,hpa_0,hpa_1,hpa_2,hpa_3,hpa_4,hpa_5,hpa_6,hpa_7,hpa_8,...,network_118,network_119,network_120,network_121,network_122,network_123,network_124,network_125,network_126,network_127
0,ENSG00000136754,1.224354,0.375444,0.415797,-0.193960,-0.575138,0.628131,-0.201646,-0.070530,-1.111441,...,-0.140911,-0.081005,-0.272773,0.103873,-0.128576,0.064615,0.496088,0.023619,0.239841,-0.381192
1,ENSG00000097007,2.301717,0.496194,0.934254,-1.010038,0.761272,0.521151,0.233309,-0.609991,-1.182277,...,-0.041017,-0.058913,0.098920,0.123309,0.102384,0.071378,-0.003177,0.035893,0.073140,-0.080192
2,ENSG00000143322,1.687926,0.165574,-0.283591,-0.589243,0.995969,0.729554,-0.216856,-0.573510,-1.171009,...,0.117642,-0.097810,0.114063,0.219740,-0.027918,0.148278,0.054275,0.041046,0.047968,0.033024
3,ENSG00000144476,0.672661,0.740794,0.482869,1.095246,0.375547,1.387898,-0.059448,-0.018398,-1.061311,...,0.301467,0.160640,0.049330,0.423391,0.094430,0.277143,-0.261512,0.187081,0.054370,0.129500
4,ENSG00000123983,1.325595,0.596590,-0.036726,0.099112,0.679536,0.678640,-0.449801,0.092542,-0.877696,...,0.125208,0.013315,-0.172672,0.149829,-0.181070,-0.067436,-0.297485,0.160468,-0.118949,-0.217159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,ENSG00000130844,1.703319,-1.538011,0.976639,0.103362,0.270447,0.698892,-0.142706,-0.054580,-0.634387,...,0.661879,0.244783,0.017495,0.030970,-0.176740,0.187825,0.161338,0.029044,0.009634,-0.083735
640,ENSG00000126746,1.703319,-1.538011,0.976639,0.103362,0.270447,0.698892,-0.142706,-0.054580,-0.634387,...,0.004761,-0.345387,-0.345399,-0.144930,0.257357,0.244146,0.001122,-0.266535,0.172343,0.143935
641,ENSG00000197013,1.503480,-1.626249,0.635482,0.246415,0.371925,-0.183920,-0.085103,0.171952,0.154054,...,0.341254,-0.096807,-0.009818,-0.342217,-0.421693,-0.176147,-0.026246,-0.320051,0.306208,-0.074802
642,ENSG00000198795,2.082088,-1.458554,1.708238,-0.042659,0.100264,0.606444,0.057277,0.026743,-0.745850,...,-0.396426,-0.434153,0.174432,-0.213453,0.346840,0.093357,0.041941,0.188002,0.067718,-0.392879


In [15]:
label_col = 'gda_score'

node_dataset = dgn.merge(master['ensembl'], on='ensembl', how='right').fillna(0)

node_dataset[label_col] = node_dataset[label_col].map({0: pd.NA, 1:1})

print('# of positive labels: ', node_dataset[label_col].sum())

node_dataset

labels = node_dataset

labels[label_col] = labels[label_col].astype('Int64')

num_neg_samples = node_dataset[label_col].sum()

unlabeled_samples = labels.loc[labels['gda_score'].fillna(0)!=1]

num_random_samplings = 100
for i in range(num_random_samplings):
    negative_samples = np.random.choice(unlabeled_samples['ensembl'], size=num_neg_samples, replace=False)
    labels[f'label_{i}'] = labels[label_col]
    labels.loc[labels['ensembl'].isin(negative_samples), f'label_{i}'] = 0
labels

# of positive labels:  96


,ensembl,gda_score,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,...,label_90,label_91,label_92,label_93,label_94,label_95,label_96,label_97,label_98,label_99
0,ENSG00000136754,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,...,<NA>,<NA>,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>
1,ENSG00000097007,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,...,0,<NA>,<NA>,<NA>,<NA>,<NA>,0,<NA>,<NA>,<NA>
2,ENSG00000143322,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,<NA>,...,<NA>,<NA>,0,<NA>,<NA>,<NA>,0,<NA>,<NA>,<NA>
3,ENSG00000144476,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>,0,<NA>,<NA>
4,ENSG00000123983,<NA>,<NA>,<NA>,<NA>,<NA>,0,<NA>,<NA>,0,...,0,<NA>,<NA>,0,<NA>,<NA>,<NA>,0,0,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,ENSG00000130844,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,...,<NA>,0,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
640,ENSG00000126746,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
641,ENSG00000197013,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
642,ENSG00000198795,<NA>,0,<NA>,<NA>,<NA>,0,<NA>,<NA>,<NA>,...,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [16]:

labels.to_csv("ESCA/eso_training_labels_trials.csv", index=None)